# Combined Companies

### Introduction

Now that we've done the work of integrating our contacts, let's integrate the information that we have on companies.

### Final results

Initially, we would like our merged company information to look like the following:

<img src="./initial_query.png" width="100%">

So we can see above, that we combined both the hubspot companies and rds companies.  

Then, from there, we should add in the additional columns from the rds companies models.

Ultimately, with these additional features added, our data will look like the following:

<img src="./combined_companies.png" width="100%">

Make sure to run the integration models to ensure that the data compiles.

### Summary

In this lesson, we worked to combined our company data from both our hubspot and rds staging models.  Once we merged the data, we then continued on to include the additional columns in the rds staging model.

### Solution

```sql
with hubspot_companies as (
    select * from {{ ref('stg_hubspot_companies') }}
), rds_companies as (
    select * from {{ ref('stg_rds_companies') }}
), merged_companies as (
    select company_id as hubspot_company_id, null as rds_company_id, name from hubspot_companies union all
    select null as hubspot_company_id, company_id as rds_company_id, name from rds_companies
), deduped as (
    select 
    max(hubspot_company_id) as hubspot_company_id,
    max(rds_company_id) as rds_company_id, name
     from merged_companies group by name
)
 select {{ dbt_utils.surrogate_key(['deduped.name']) }} as company_pk, hubspot_company_id, rds_company_id, deduped.name, address,
   postal_code, city, country from deduped
  left outer join rds_companies on rds_companies.company_id = deduped.rds_company_id
```